In [ ]:
!pip install gym[box2d] box2d-py swig gymnasium stable_baselines3[extra] box2d box2d-kengz ipywidgets ffmpeg

In [ ]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy

# Create environment
env = gym.make("LunarLander-v2", render_mode="rgb_array")

# Instantiate the agent
model = PPO(
    'MlpPolicy',
    env,
    verbose=1,
    ##### HYPERPARAMETERS HERE!!!!
    learning_rate=0.001,
    batch_size=32,
    )

# Train the agent and display a progress bar
model.learn(total_timesteps=int(10000), progress_bar=True)

# Save the agent
model.save("ppo_lunar")
#del model  # delete trained model to demonstrate loading

In [ ]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
import imageio
import numpy as np

# Load the trained agent
env = gym.make("LunarLander-v2", render_mode="rgb_array")
model = PPO.load("ppo_lunar", env=env)

# Evaluate the agent
mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10)
print('Mean reward:', mean_reward, 'Std. reward:', std_reward)

# Test the trained agent
images = []
j = 0
vec_env = model.get_env()
obs = vec_env.reset()
img = model.env.render(mode="rgb_array")
for i in range(2000):
    images.append(img)
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, dones, info = vec_env.step(action)
    #vec_env.render()
    if dones:
        j = j + 1
        print('Episode:', j, 'Rewards:', rewards, 'Info:', info, 'Dones:', dones)
        imageio.mimsave(f"./images/ppo_lander_{j}.gif", [np.array(img) for k, img in enumerate(images) if k%2 == 0], fps=29)
        images = []
    img = model.env.render(mode="rgb_array")
vec_env.close()